In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('../input/reddit-mental-health-data/Reddit Dataset/final_datasets/final_train.csv')
df_val = pd.read_csv('../input/reddit-mental-health-data/Reddit Dataset/final_datasets/final_val.csv')
df_test = pd.read_csv('../input/reddit-mental-health-data/Reddit Dataset/final_datasets/final_test.csv')

In [ ]:
from transformers import AlbertTokenizer, AlbertModel
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
#model = AlbertModel.from_pretrained('roberta-base')
#text = "Replace me by any text you'd like."
#encoded_input = tokenizer(text, return_tensors='pt')
#output = model(**encoded_input)

In [ ]:
import torch

labels = {'adhd':0,
          'anxiety':1,
          'bipolar':2,
          'depression':3,
          'ptsd':4,
          'none':5
          }

# Dataset

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df, max_length, input_field):

        self.labels = [labels[label] for label in df['class_name']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = max_length, truncation=True,
                                return_tensors="pt") for text in df[input_field]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

# Model

In [ ]:
from torch import nn
from transformers import AlbertTokenizer, AlbertModel

class AlbertClassifier(nn.Module):

    def __init__(self, dropout=0.3):

        super(AlbertClassifier, self).__init__()

        self.albert = AlbertModel.from_pretrained('albert-base-v2')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 6)
        self.softmax = nn.Softmax()

    def forward(self, input_id, mask):

        output = self.albert(input_ids= input_id, attention_mask=mask)
        last_hidden_layer = output.last_hidden_state
        del output
        pooled_output = torch.mean(last_hidden_layer, dim = 1)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.softmax(linear_output)

        return final_layer

# Train

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, df_train, df_val, learning_rate, epochs, batch_size, max_length, input_field):
    
    train, val = Dataset(df_train, max_length, input_field), Dataset(df_val, max_length, input_field)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].squeeze(1).to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(df_train): .3f} \
                | Train Accuracy: {total_acc_train / len(df_train): .3f} \
                | Val Loss: {total_loss_val / len(df_val): .3f} \
                | Val Accuracy: {total_acc_val / len(df_val): .3f}')
    
    PATH = "albert" + input_field + ".pt"
    torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, PATH)

# Evaluate

In [ ]:
def evaluate_posts(model, df_test, max_length, input_field):
    
    test = Dataset(df_test, max_length, input_field)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].squeeze(1).to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')

# Training Model on Posts

In [ ]:
epochs = 5
model = AlbertClassifier()
lr = 1e-5
batch_size = 16

train(model, df_train, df_val, lr, epochs, batch_size, 512, 'post')


#evaluate_posts(model, df_test, 512, 'post')

# Training Model on Titles

In [ ]:
epochs = 5
model = AlbertClassifier()
lr = 1e-5
batch_size = 32

train(model, df_train, df_val, lr, epochs, batch_size, 35, 'title')

#evaluate_posts(model, df_test, 35, 'title')

# Training Model on Titles + Posts

In [ ]:
epochs = 5
model = AlbertClassifier()
lr = 1e-5
batch_size = 16

train(model, df_train, df_val, lr, epochs, batch_size, 512, 'title_post')

#evaluate_posts(model, df_test, 512, 'title_post')